In [1]:
!pip install transformers[sentencepiece] evaluate datasets nltk tqdm rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 52.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8827bdc7c8d34774c5212840087d01aaebc5e49975a39d3515ce086257dc9e

In [2]:
from datasets import load_dataset

spanish_dataset = load_dataset("amazon_reviews_multi", "es")
english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [6]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


show_samples(english_dataset)


'>> Title: Pole Is not 1" in Diameter!!!!!!!'
'>> Review: If the pole was a little smaller than 1" in diameter, it would have been perfect. Bought weight plates 1" in diameter and could not put them on the pole. Such a disappointment. Now I have to buy 2" diameter plates or simply send the sled back. Why would the sled not be a tad bit smaller than 1" when weights plates are the primary source for adding weight to the system...........ughhhh!!!!!!!!!!!!!!!'

'>> Title: Not a Good Choice for Outdoor Target Shooting'
'>> Review: These arrows are in fact made of carbon fiber and have a compression fit tip. If you shoot outdoors and miss your soft target, you are unlikely to retrieve your arrow without destroying it. For example, if the arrow gets stuck in a tree or even in the ground, pulling it out results in the arrow head being left behind. The compression fitting is just not that strong. I also noticed when shooting on very cold days that the arrow shafts developed very small cracks 

In [4]:
english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20]

home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: product_category, dtype: int64

In [5]:
english_dataset.reset_format()

In [6]:
show_samples(english_dataset)


'>> Title: Worked in front position, not rear'
'>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.'

'>> Title: meh'
'>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue'

'>> Title: Can't beat these for the money'
'>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. 

In [3]:
train_size = 20000
test_size = int(0.1 * train_size)

english_dataset = english_dataset["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
english_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 2000
    })
})

In [4]:
train_size = 20000
test_size = int(0.1 * train_size)

spanish_dataset = spanish_dataset["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
spanish_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 2000
    })
})

In [7]:
from datasets import concatenate_datasets, DatasetDict

mix_dataset = DatasetDict()

for split in english_dataset.keys():
    mix_dataset[split] = concatenate_datasets(
        [english_dataset[split], spanish_dataset[split]]
    )
    mix_dataset[split] = mix_dataset[split].shuffle(seed=42)

# Peek at a few examples
show_samples(mix_dataset)


'>> Title: Very sweet'
'>> Review: Tried to use this syrup to make a sangria. Too thick and sweet for this application.'

'>> Title: Simple to use'
'>> Review: This bra is way easier to use than my medela or lansinoh hands free bras! I can put it OVER my clothing and pump! Very simple, and works effectively.'

'>> Title: Two Stars'
'>> Review: Didn't slow the barking even a little bit.'


In [8]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [336, 259, 28387, 11807, 287, 62893, 295, 12507, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
['▁I', '▁', 'loved', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '</s>']

['▁I', '▁', 'loved', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '</s>']

In [11]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["review_title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_datasets = mix_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [13]:
import evaluate

rouge_score = evaluate.load("rouge")

In [14]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(mix_dataset["train"][1]["review_body"]))

Wires came out within a month.


In [18]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]
    return metric.compute(predictions=summaries, references=dataset["review_title"])

In [19]:
import pandas as pd

score = evaluate_baseline(mix_dataset["test"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 13.12, 'rouge2': 6.43, 'rougeL': 12.35, 'rougeLsum': 12.54}

In [20]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [20]:
tokenized_datasets = tokenized_datasets.remove_columns(
    mix_dataset["train"].column_names
)

In [21]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [24]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [25]:
model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, epochs=8
)

Epoch 1/8
5000/5000 [==============================] - 1462s 285ms/step - loss: 7.0418 - val_loss: 3.3351
Epoch 2/8
5000/5000 [==============================] - 1417s 283ms/step - loss: 4.1365 - val_loss: 3.0821
Epoch 3/8
5000/5000 [==============================] - 1416s 283ms/step - loss: 3.5796 - val_loss: 2.9715
Epoch 4/8
5000/5000 [==============================] - 1418s 284ms/step - loss: 3.2966 - val_loss: 2.8924
Epoch 5/8
5000/5000 [==============================] - 1419s 284ms/step - loss: 3.1408 - val_loss: 2.8455
Epoch 6/8
5000/5000 [==============================] - 1417s 283ms/step - loss: 3.0556 - val_loss: 2.8146
Epoch 7/8
5000/5000 [==============================] - 1415s 283ms/step - loss: 3.0000 - val_loss: 2.7907
Epoch 8/8
5000/5000 [==============================] - 1416s 283ms/step - loss: 2.9729 - val_loss: 2.7850


In [26]:
model.save_pretrained('drive/MyDrive/Model/Multilingual_Summarisation_Model')

In [26]:
model = TFAutoModelForSeq2SeqLM.from_pretrained('drive/MyDrive/Model/Multilingual_Summarisation_Model')

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at drive/MyDrive/Model/Multilingual_Summarisation_Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [48]:
from tqdm import tqdm
import numpy as np
import tensorflow as tf

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=512
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)


@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=128,
    )


all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 500/500 [01:54<00:00,  4.38it/s]


In [49]:
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value * 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 6.6964, 'rouge2': 0.0, 'rougeL': 6.6964, 'rougeLsum': 6.6964}

In [31]:
from transformers import pipeline

local = "drive/MyDrive/Model/Multilingual_Summarisation_Model"
summarizer = pipeline("summarization", model=local, tokenizer = tokenizer)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at drive/MyDrive/Model/Multilingual_Summarisation_Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [32]:
def print_summary(idx):
    review = mix_dataset["test"][idx]["review_body"]
    title = mix_dataset["test"][idx]["review_title"]
    summary = summarizer(mix_dataset["test"][idx]["review_body"])[0]["summary_text"]
    print(f"'>>> Review: {review}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Summary: {summary}'")

In [39]:
print_summary(2241)

'>>> Review: A bit light for Fall. Overall a very good fit and design, nice looking. Haven't worn for the ride yet because a bit chilly in the morning for short sleeves.'

'>>> Title: Cool and nice'

'>>> Summary: Very good fit and design'


In [46]:
summarizer("I bought the product because the product ratings were very high but it almost instantly stopped working after a few days and moreover it smelt bad too.")[0]["summary_text"]

'Poor quality'

In [47]:
summarizer("I bought the product because the product ratings were very high and it actually did meet expectations. It has been of great use and i carry it everywhere with me")[0]["summary_text"]

'Great product'